In [10]:
import numpy as np
from scipy.optimize import minimize
import math 
# Define constants
L = 1852 * 4
gamma = np.sin(np.radians(31.5)) / (np.sin(np.radians(31.5)) + np.sin(np.radians(28.5)))
alpha = np.radians(1.5)
theta = np.radians(120)

def objective(z):
    return math.sqrt(len(z)**2)

def constraint1(z):
    y = np.cumsum(z)
    D = (2 * 1852 * np.tan(np.radians(1.5)) + 110) - y * np.tan(np.radians(1.5))
    W = (np.sin(theta) * (np.cos(alpha) ** 2)) / (np.cos(theta / 2 + alpha) * np.cos(theta / 2 - alpha)) * D
    y_next=np.append(y[1:],0)
    W_next=np.append(W[1:],0)
    delta=y+W*(1-gamma)-(y_next-W_next*gamma)
    return sum(W)-sum(delta)-L

def constraint2(z):
    y = np.cumsum(z)
    D_n = (2*1852*np.tan(alpha)+110)-y[-1]*np.tan(alpha)
    W_n=(np.sin(theta)*(np.cos(alpha)**2))/(np.cos(theta/2+alpha)*np.cos(theta/2-alpha))*D_n
    return L-y[-1]-W_n*(1-gamma)

constraints=[{'type':'eq','fun':constraint1},
             {'type':'eq','fun':constraint2}]

# Initial guess for z: an array of ones.
n0=10 # initial guess for number of variables in 'z'
z0=np.ones(n0)

# Bounds for z: each element must be between 0 and L.
bounds_z=[(0,L)]*n0

result=minimize(objective,z0,method='SLSQP',bounds=bounds_z,constraints=constraints,options={'disp':True})

print('The minimum value of n is:',len(result.x))


Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: 10.0
            Iterations: 33
            Function evaluations: 551
            Gradient evaluations: 29
The minimum value of n is: 10


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from Q3_BasicMath import calculate_Wn_1
from math import *
from scipy.optimize import fsolve

import random
# 用于记录每一个y_i的位置
line_positions = [0]
L = 1852 * 4
gamma = sin(np.radians(31.5)) / (sin(np.radians(31.5)) + sin(np.radians(28.5)))
step = 0.02*1852
simulate_times = 100

# 设置宽度左右两边的比例关系
west_width_percent = sin(radians(61.5))/(sin(radians(61.5))+sin(radians(58.5)))

# 设置计算出来的宽度
def detect_width(y_position):
    return calculate_Wn_1(y_position)


# 从前向后算
def Solvevalue_1(unsolvedvalue,edge):
    y = unsolvedvalue[0]
    edge = edge
    return [
        detect_width(y)*gamma - y + edge,
    ]
def Calculate_lines_1(y):
    line_positions[0]=y
    while line_positions[-1] <= L:
        width = detect_width(line_positions[-1])
        edge = line_positions[-1]+width*(1-gamma-0.1)
        # print(detect_width(edge)*gamma - edge)
        Solvevalue_1([0],edge)
        y_next = fsolve(Solvevalue_1,[0],args=edge)
        line_positions.append(y_next[0])
    temp1 = line_positions[len(line_positions)-2]
    temp2 = line_positions[-1]
    flag_last = temp1 + detect_width(temp1)*(1-gamma)
    flag_next = temp2 - detect_width(temp2)*(gamma)
    ita = (flag_last-flag_next)/ detect_width(temp1)
    return ita,temp1,temp2,flag_last,flag_next

# 从后向前算
def Solvevalue_2(unsolvedvalue,edge):
    y = unsolvedvalue[0]
    edge = edge
    return [
        detect_width(y)*(1-gamma) + y - edge,
    ]
def Calculate_lines_2(y):
    line_positions[0]=y
    while line_positions[-1] >= 0:
        width = detect_width(line_positions[-1])
        edge = line_positions[-1]-width*(gamma-0.1)
        print(edge)
        Solvevalue_2([0],edge)
        y_next = fsolve(Solvevalue_2,[0],args=edge)
        line_positions.append(y_next[0])
    temp1 = line_positions[len(line_positions)-2]
    temp2 = line_positions[-1]
    flag_last = temp1 - detect_width(temp1)*gamma
    flag_next = temp2 + detect_width(temp2)*(1-gamma)
    ita = (flag_next-flag_last)/ detect_width(temp1)
    return ita,temp1,temp2,flag_last,flag_next


Solvevalue_1([0],0)
solved = fsolve(Solvevalue_1,[0],args=0)
print(solved)
print(Calculate_lines_1(solved[0]))
plt.figure(figsize=(10,5))
plt.plot(line_positions)

line_positions=[0]
Solvevalue_2([0],L)
solved = fsolve(Solvevalue_2,[0],args=L)
print(solved)
print(Calculate_lines_2(solved[0]))
plt.figure(figsize=(10,5))
plt.plot(line_positions)

ModuleNotFoundError: No module named 'Q3_BasicMath'